<a href="https://colab.research.google.com/github/Alexcmbs/test-stage-m/blob/main/test_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Test de LLM sur google collab

Mistral 7b


In [11]:
!pip install -q -U langchain transformers bitsandbytes accelerate

In [19]:
import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
cannot import name 'is_torch_xla_available' from 'transformers.utils' (/usr/local/lib/python3.10/dist-packages/transformers/utils/__init__.py)

In [4]:

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model_4bit = AutoModelForCausalLM.from_pretrained( "mistralai/Mistral-7B-Instruct-v0.1", device_map="auto",quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [11]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Update the model identifier here
model_4bit = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    device_map="auto",
    quantization_config=quantization_config,
)

# Update the tokenizer identifier here
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Assurez-vous que la configuration de quantification est compatible avec meta-llama/Llama-2-13b, sinon ajustez ou retirez.
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Remplacez par l'identifiant de modèle approprié
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-13b",
    device_map="auto",
    quantization_config=quantization_config,  # Si applicable
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b")


NameError: name 'BitsAndBytesConfig' is not defined

In [13]:

pipeline_inst = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=2500,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

In [12]:
llm = HuggingFacePipeline(pipeline=pipeline_inst)

In [20]:

template = """[INST] You are an respectful and helpful assistant, respond always be precise, assertive and politely answer in few words conversational french.
Answer the question below from context below in 50 characters :
{question} [/INST]
"""

In [15]:

def generate_response(question):
  prompt = PromptTemplate(template=template, input_variables=["question","context"])
  llm_chain = LLMChain(prompt=prompt, llm=llm)
  response = llm_chain.run({"question":question})
  return response


In [9]:

generate_response("context : Le chiffre d’affaires brut d’Antarius s’eleve a 1 606 M a fin 2022, en leger recul de -5 M par rapport a 2021.En assurance vie epargne, la collecte brute atteint 1 554 M sur l’exercice 2022, en baisse de 7 M par rapport a l’exercice prece-dent. Apres le fort rebond du marche de l’assurance vie en 2021, la guerre en Ukraine et ses repercussions macro-economiques ont contribue au ralentissement de la collecte sur le marche francais (-3 %) en 2022. Dans ce contexte, Antarius a realise une bonne performance commerciale, la baisse de la collecte brute  etant attribuable aux supports en unites de compte (-11 M). La  part des supports en unites de compte dans la collecte globale  represente 36 % en 2022, a un niveau stable par rapport a 2021. question : peux tu me parler de la situation financière d'Antarius.")

NameError: name 'generate_response' is not defined